#### Analysis

In [28]:
bnum = 8
corr_all_Tsamples = []
secret_range = 2**(bnum)
M = round(secret_range / 2)
hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]

In [30]:
mult_result = (256 * 10) % secret_range
print(hw[mult_result])

0


In [2]:
import scipy
import numpy as np

In [26]:
def hamming_weight(x, is_int = True):
    if not is_int:
        return np.count_nonzero(x == 1)
    return bin(x).count("1")
def get_overflowed_val(val = 0, bnum = 8):
    border = 2**(bnum-1)
    if val >= border:
        val = val % (border*2)
        if val >= border:
            val = -border + (val - border)
    return val
def V2_ANN_CPA_OTS(known_input : list, waves = np.array([]), n_traces = 300, trace_len = 24000, ith_weight = 0): # Vanilla ANN CPA, for one time_sample a
    bnum = 8
    corr_all_Tsamples = []
    secret_range = 2**(bnum)
    M = round(secret_range / 2)
    hw = [hamming_weight(secret_value) for secret_value in range(secret_range)]

    qsum_L_list = []
    l_diff_list = []
    for time_sample in range(trace_len):
      L = waves[:,time_sample]
      L_mean = np.mean(L, axis = 0)
      qsum_L = 0
      l_diff = [l - L_mean for l in L]
      for w in range(M*2):
        qsum_L += (l_diff[w])**2
      qsum_L_list.append(qsum_L)
      l_diff_list.append(l_diff)

    # wave_Blen = round(len(waves[0]))
    for secret_value in trange(secret_range, desc='Calculating Correlations for secret key'): # For current WeightHyphothesis do
        corr_Tsamples = []
        H = [] #Hyphotheticalleakage
        for j in range(n_traces):  #  Get an array of possible HammingWeights for current WeightHyphothesis (for all the traces)
            mult_result = (secret_value * known_input[j]) % secret_range
            H.append(hw[mult_result])
            #H.append(mult_result)
        H_mean = np.mean(H, axis = 0)
        h_diff = [h - H_mean for h in H]
        qsum_H = 0
        for w in range(M*2):
          qsum_H += (h_diff[w])**2
        # For current WeightHyphothesis, create an Correlation, this vector needs to be created for each time sample:
        for time_sample in range(trace_len):
            L = waves[:,time_sample]
            l_diff = l_diff_list[time_sample]
            sum_HL = 0
            for w in range(M*2):
              sum_HL += (h_diff[w])*(l_diff[w])
            divider = (qsum_H**(1/2))*(qsum_L_list[time_sample]**(1/2))
            corr = 0
            if divider != 0:
              corr = sum_HL / divider
            corr_Tsamples.append(abs(corr))
        corr_all_Tsamples.append(corr_Tsamples)
    return corr_all_Tsamples
def plot_mult(corr_list, type = "norm"):
  t_corr_list = corr_list.transpose()
  import matplotlib.pyplot as plt
  image = plt.figure()
  for corr in corr_list:
    plt.plot(corr)
  plt.show()
  image.savefig(f'./figures/{epochs}_{type}_corr.png')


def plot_one(array, type = "norm"):
  import matplotlib.pyplot as plt
  image = plt.figure()
  plt.plot(array)
  plt.show()
  image.savefig(f'./figures/{epochs}_{type}_corr.png')

def createDiffWave(waves, name):
        diff_waves = []
        square_waves = []
        avg_wave = np.mean(waves,axis=0)
        
        for i in waves:
            diff_waves.append(np.subtract(i,avg_wave))
        for i in diff_waves:
            square_wave = []
            for ii in i:
                square_wave.append(ii**2)
            square_waves.append(square_wave)
        
        diff_avg = np.mean(diff_waves,axis=0)
        var = np.mean(square_waves,axis=0)
    
        plot_one(np.array(waves[0]), type = f"wave0_{name}")
        plot_one(np.array(avg_wave), type = f"avg_wave_{name}")
        plot_one(np.array(diff_avg), type = f"difference_wave_{name}")
        plot_one(np.array(var), type = f"variation_{name}")
        return diff_avg, avg_wave, var, waves

In [27]:
def get_weight(correlation_matrix: np.array, peak_range = 0.3, show_info=False, lastIndex = 0, map_2dgrMaxs = None, mountain_half_dist = 20, level = 5):
    #FIND index of highest value for each column (correlation of time sample)
    indx_ColmnMax = correlation_matrix.argmax(axis=0) #Return an vector of indexes of max values in each column of matrix
    
    #CREATE an array from previous finds
    colmn_MaxVector = np.array([correlation_matrix[indx_ColmnMax[i]][i] for i in range(len(indx_ColmnMax))]) #Return a vector of max values for each column in matrix

    #FIND GLOBAL MAX
    indx_globMax = np.argmax(colmn_MaxVector)
    val_globMax = colmn_MaxVector[indx_globMax]
    vec_len = len(colmn_MaxVector)


    if map_2dgrMaxs is None:

        vals_locMaxVector0 = []
        tmp_indices = []
        for i in range(len(colmn_MaxVector)):
            tmp_indices.append(i)
            vals_locMaxVector0.append(0)
        for i in range(level):
            tmp_array = []
            for indx in tmp_indices:
                tmp_array.append(colmn_MaxVector[indx])
            indices, props = scipy.signal.find_peaks(tmp_array)
            for indx in range(len(indices)):
                indices[indx] = tmp_indices[indices[indx]]
            tmp_indices = indices
        for indx in tmp_indices:
            if val_globMax-peak_range <= colmn_MaxVector[indx]:
                vals_locMaxVector0[indx] = colmn_MaxVector[indx]
        
        vals_locMaxVector1 = []
        #FIND 1. local maxims and zero-out all values in tresshold range to global_correlation
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = colmn_MaxVector[indx]
            val_toAppend = 0
            if (value + peak_range) >= val_globMax and ((indx+1) < vec_len) and (value > colmn_MaxVector[indx+1]):
                val_toAppend = value
            vals_locMaxVector1.append(val_toAppend)
        secret_corr = 0 #FINAL correlation
        secret_colmn = 0 #Column/Time sample of Final correlation
    
        #FIND 2. local maxims
        vals_locMaxVector2 = []
        last_val = vals_locMaxVector1[0]
        last_indx = 0
        zero_counter = 0
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = vals_locMaxVector1[indx]
            vals_locMaxVector2.append(0)
            if value != 0:#Cross the desert, and on start of the next mountain, reminiscence about last mountain and save it to the memoar 
                if zero_counter > mountain_half_dist:
                    vals_locMaxVector2[last_indx] = last_val
                    #Reset
                    last_val = 0
                    last_indx = indx
                else:
                    if last_val < value:
                        last_val = value
                        last_indx = indx
                zero_counter = 0
            else:
                zero_counter += 1
        vals_locMaxVector2[last_indx] = last_val



        #FIND 3. local maxims
        vals_locMaxVector3 = []
        last_val = vals_locMaxVector1[0]
        last_indx = 0
        zero_counter = 0
        for indx in range(vec_len): #shoud create vector with values which are local-maxims, other values are zeroed out
            value = vals_locMaxVector1[indx]
            vals_locMaxVector3.append(0)
            if value > (last_val-peak_range):#Cross the desert, and on start of the next mountain, reminiscence about last mountain and save it to the memoar 
                if zero_counter > mountain_half_dist:
                    vals_locMaxVector3[last_indx] = last_val
                    #Reset
                    last_val = 0
                    last_indx = indx
                else:
                    if  last_val > value:
                        vals_locMaxVector3[last_indx] = last_val
                    last_val = value
                    last_indx = indx
                zero_counter = 0
            else:
                zero_counter += 1
        vals_locMaxVector3[last_indx] = last_val
    else:
        vals_locMaxVector2 = map_2dgrMaxs
    
    secret_corr = 0 #FINAL correlation
    secret_colmn = 0 #Column/Time sample of Final correlation
    nextIndex = lastIndex
    found = False
    print(f"In range {lastIndex} - {len(indx_ColmnMax)}")
    for indx in range(lastIndex, len(indx_ColmnMax)): #Find first local maxim in the peak_range from global maxim
        value = vals_locMaxVector2[indx]
        if value != 0:
            if not found:
                secret_corr = value
                secret_colmn = indx
                found = True
                continue
            nextIndex =  indx - round((indx - secret_colmn) / 2)
            break
    secret_val = indx_ColmnMax[secret_colmn]



    
    if show_info and map_2dgrMaxs is None:    
        #fig = plt.figure()
        #plt.plot(indx_ColmnMax)
        #plt.title('Indexes of max row in column')
        
        #fig = plt.figure()
        #plt.plot(colmn_MaxVector)
        #plt.title('Max correlation values grapth')



        print(f" Index of global max: {indx_globMax} and global_max_val: {val_globMax}")


        fig = plt.figure()
        plt.plot(vals_locMaxVector0, color="green")
        plt.title(f'{level}th-level degree Local maxims graph')
        plt.xlabel("Time Sample")
        plt.ylabel("Secret Value")
        plt.show()

        indx_localMax = []
        for i in range(len(vals_locMaxVector0)):
            val_toAppend = 0
            if vals_locMaxVector0[i] > 0:
                val_toAppend = indx_ColmnMax[i]
            indx_localMax.append(val_toAppend)
        print(f"Local maxims: {[int(indx_ColmnMax[i]) for i in range(len(vals_locMaxVector0)) if vals_locMaxVector0[i] > 0]}")
        fig = plt.figure()
        plt.plot(indx_localMax, color="green")
        plt.title("0. Local maxim's indexes graph")
        plt.xlabel("Time Sample")
        plt.ylabel("Index")
        plt.show()
        

        
        fig = plt.figure()
        plt.plot(vals_locMaxVector1, color="red")
        plt.title('1-th degree Local maxims graph')
        plt.xlabel("Time Sample")
        plt.ylabel("Secret Value")
        plt.show()
        
        indx_localMax = []
        for i in range(len(vals_locMaxVector1)):
            val_toAppend = 0
            if vals_locMaxVector1[i] > 0:
                val_toAppend = indx_ColmnMax[i]
            indx_localMax.append(val_toAppend)
        print(f"Local maxims: {[int(indx_ColmnMax[i]) for i in range(len(vals_locMaxVector1)) if vals_locMaxVector1[i] > 0]}")
        fig = plt.figure()
        plt.plot(indx_localMax, color="red")
        plt.title("1. Local maxim's indexes graph")
        plt.xlabel("Time Sample")
        plt.ylabel("Index")
        plt.show()       



        fig = plt.figure()
        plt.plot(vals_locMaxVector2, color="blue")
        plt.title('2-th degee Local maxims graph')
        plt.xlabel("Time Sample")
        plt.ylabel("Secret Value")
        plt.show()


        indx_localMax = []
        for i in range(len(vals_locMaxVector2)):
            val_toAppend = 0
            if vals_locMaxVector2[i] > 0:
                val_toAppend = indx_ColmnMax[i]
            indx_localMax.append(val_toAppend)
        print(f"Local maxims: {[int(indx_ColmnMax[i]) for i in range(len(vals_locMaxVector2)) if vals_locMaxVector2[i] > 0]}")
        fig = plt.figure()
        plt.plot(indx_localMax, color="blue")
        plt.title("2. Local maxim's indexes graph")
        plt.xlabel("Time Sample")
        plt.ylabel("Index")
        plt.show()




        fig = plt.figure()
        plt.plot(vals_locMaxVector3, color="purple")
        plt.title('2-th-var degree Local maxims graph')
        plt.xlabel("Time Sample")
        plt.ylabel("Secret Value")
        plt.show()

        indx_localMax = []
        for i in range(len(vals_locMaxVector3)):
            val_toAppend = 0
            if vals_locMaxVector3[i] > 0:
                val_toAppend = indx_ColmnMax[i]
            indx_localMax.append(val_toAppend)
        print(f"Local maxims: {[int(indx_ColmnMax[i]) for i in range(len(vals_locMaxVector3)) if vals_locMaxVector3[i] > 0]}")
        fig = plt.figure()
        plt.plot(indx_localMax, color="purple")
        plt.title("3. Local maxim's indexes graph")
        plt.xlabel("Time Sample")
        plt.ylabel("Index")
        plt.show()


    
    print(f" Found the soonest secret value is {secret_val}, at the time sample {secret_colmn}, with correlation {secret_corr = :.3f}")
    return secret_val, secret_colmn, secret_corr, vals_locMaxVector2, colmn_MaxVector, nextIndex, vals_locMaxVector2
#results_directory = {}
#secret_value, time_sample, correlation, localMax_vector = get_weight(np.array(ncorr_all), show_info=True)

In [7]:
print("✔️ The Analyser succesfuly runned.")

✔️ The Analyser succesfuly runned.
